In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
cd drive/MyDrive/FrenetikusNyelvtech/

/content/drive/MyDrive/FrenetikusNyelvtech


In [ ]:
pwd

'/content/drive/MyDrive/FrenetikusNyelvtech'

In [ ]:
!tar -xvf  'data.tar.gz' 

._data
data/
data/claims_dev.jsonl
data/cross_validation/
data/claims_train.jsonl
data/claims_test.jsonl
data/corpus.jsonl
data/cross_validation/._fold_4
data/cross_validation/fold_4/
data/cross_validation/fold_3/
data/cross_validation/fold_2/
data/cross_validation/fold_5/
data/cross_validation/._fold_1
data/cross_validation/fold_1/
data/cross_validation/fold_1/claims_dev_1.jsonl
data/cross_validation/fold_1/claims_train_1.jsonl
data/cross_validation/fold_5/claims_train_5.jsonl
data/cross_validation/fold_5/claims_dev_5.jsonl
data/cross_validation/fold_2/claims_dev_2.jsonl
data/cross_validation/fold_2/claims_train_2.jsonl
data/cross_validation/fold_3/claims_train_3.jsonl
data/cross_validation/fold_3/claims_dev_3.jsonl
data/cross_validation/fold_4/claims_dev_4.jsonl
data/cross_validation/fold_4/claims_train_4.jsonl


In [ ]:
pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.7/dist-packages (1.1.0)


In [ ]:
ls

 data/
 data.tar.gz
'Learning from data - a short course (Abu-Mostafa et al, 2012).pdf'
'Nyeltech nagy bead.ipynb'


In [ ]:
#cat /content/data/corpus.jsonl

In [ ]:
import json

In [ ]:
claims = []
with open('data/claims_train.jsonl', 'r') as f:
    for line in f:
        claim = json.loads(line)
        claims.append(claim)
claims_by_id = {claim['id']:claim for claim in claims}
print(claims[0]['id'])

0


In [ ]:
corpus = {}
with open('data/corpus.jsonl') as f:
    for line in f:
        abstract = json.loads(line)
        corpus[str(abstract["doc_id"])] = abstract

In [ ]:
print(claims[0])

{'id': 0, 'claim': '0-dimensional biomaterials lack inductive properties.', 'evidence': {}, 'cited_doc_ids': [31715818]}


In [ ]:
!pip install jsonlines

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance
import numpy as np
import jsonlines

In [ ]:
K=2

In [ ]:
    corpus_texts = []
    corpus_ids = []
    for k, v in corpus.items():
        original_sentences = [v['title']] + v['abstract']
        processed_paragraph = " ".join(original_sentences)
        corpus_texts.append(processed_paragraph)
        corpus_ids.append(k)
    vectorizer = TfidfVectorizer(stop_words='english',
                                 ngram_range=(1, 2))
    corpus_ids = np.array(corpus_ids)
    corpus_vectors = vectorizer.fit_transform(corpus_texts)
    
    claim_vectors = vectorizer.transform([claim['claim'] for claim in claims])
    similarity_matrix = np.dot(corpus_vectors, claim_vectors.T).todense()
    
    #k = args.k_retrieval
    k = K
    orders = np.argsort(similarity_matrix,axis=0)
    retrieved_corpus = {claim["id"]: corpus_ids[orders[:,i][::-1][:k]].squeeze() for i, claim in enumerate(claims)} 
    
    with jsonlines.open("/content/claim_retrieve", 'w') as output:
        claim_ids = sorted(list(claims_by_id.keys()))
        for id in claim_ids:
            claims_by_id[id]["retrieved_doc_ids"] = retrieved_corpus[id].tolist()
            output.write(claims_by_id[id])

Retrieved_doc_id = a legfőbb K olyan abstract id amiben lehet fontos info

In [ ]:
print(retrieved_corpus)

{0: array(['10608397', '10607877'], dtype='<U9'), 2: array(['13734012', '17333231'], dtype='<U9'), 4: array(['7165938', '1387104'], dtype='<U9'), 6: array(['23117378', '2613775'], dtype='<U9'), 9: array(['44265107', '24700152'], dtype='<U9'), 10: array(['25510546', '13780287'], dtype='<U9'), 11: array(['25510546', '13780287'], dtype='<U9'), 12: array(['11705328', '43557480'], dtype='<U9'), 14: array(['1373287', '6421792'], dtype='<U9'), 15: array(['24766509', '8891333'], dtype='<U9'), 17: array(['7662395', '17450673'], dtype='<U9'), 18: array(['7165938', '1387104'], dtype='<U9'), 19: array(['36178047', '13072113'], dtype='<U9'), 20: array(['36178047', '13072113'], dtype='<U9'), 21: array(['41493639', '14021596'], dtype='<U9'), 22: array(['25451374', '35022568'], dtype='<U9'), 24: array(['1900152', '7975937'], dtype='<U9'), 25: array(['23117378', '2613775'], dtype='<U9'), 26: array(['4336849', '38410121'], dtype='<U9'), 27: array(['4336849', '38410121'], dtype='<U9'), 28: array(['280155

In [ ]:
print(corpus['10608397']['abstract'])

['BACKGROUND Paralysis or amputation of an arm results in the loss of the ability to orient the hand and grasp, manipulate, and carry objects, functions that are essential for activities of daily living.', 'Brain-machine interfaces could provide a solution to restoring many of these lost functions.', 'We therefore tested whether an individual with tetraplegia could rapidly achieve neurological control of a high-performance prosthetic limb using this type of an interface.   \n', 'METHODS We implanted two 96-channel intracortical microelectrodes in the motor cortex of a 52-year-old individual with tetraplegia.', 'Brain-machine-interface training was done for 13 weeks with the goal of controlling an anthropomorphic prosthetic limb with seven degrees of freedom (three-dimensional translation, three-dimensional orientation, one-dimensional grasping).', "The participant's ability to control the prosthetic limb was assessed with clinical measures of upper limb function.", 'This study is regis

Sentence-BERT

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

In [ ]:
print(claims)

[{'id': 0, 'claim': '0-dimensional biomaterials lack inductive properties.', 'evidence': {}, 'cited_doc_ids': [31715818], 'retrieved_doc_ids': ['10608397', '10607877']}, {'id': 2, 'claim': '1 in 5 million in UK have abnormal PrP positivity.', 'evidence': {'13734012': [{'sentences': [4], 'label': 'CONTRADICT'}]}, 'cited_doc_ids': [13734012], 'retrieved_doc_ids': ['13734012', '17333231']}, {'id': 4, 'claim': '1-1% of colorectal cancer patients are diagnosed with regional or distant metastases.', 'evidence': {}, 'cited_doc_ids': [22942787], 'retrieved_doc_ids': ['7165938', '1387104']}, {'id': 6, 'claim': '10% of sudden infant death syndrome (SIDS) deaths happen in newborns aged less than 6 months.', 'evidence': {}, 'cited_doc_ids': [2613775], 'retrieved_doc_ids': ['23117378', '2613775']}, {'id': 9, 'claim': '32% of liver transplantation programs required patients to discontinue methadone treatment in 2001.', 'evidence': {'44265107': [{'sentences': [15], 'label': 'SUPPORT'}]}, 'cited_doc_i

In [ ]:
print(len(corpus[claims[0]['retrieved_doc_ids'][0]]['abstract']))

15


In [ ]:
print(corpus[claims[1]['retrieved_doc_ids'][0]])

{'doc_id': 13734012, 'title': 'Prevalent abnormal prion protein in human appendixes after bovine spongiform encephalopathy epizootic: large scale survey', 'abstract': ['OBJECTIVES To carry out a further survey of archived appendix samples to understand better the differences between existing estimates of the prevalence of subclinical infection with prions after the bovine spongiform encephalopathy epizootic and to see whether a broader birth cohort was affected, and to understand better the implications for the management of blood and blood products and for the handling of surgical instruments.   \n', 'DESIGN Irreversibly unlinked and anonymised large scale survey of archived appendix samples.   \n', 'SETTING Archived appendix samples from the pathology departments of 41 UK hospitals participating in the earlier survey, and additional hospitals in regions with lower levels of participation in that survey.   ', 'SAMPLE 32,441 archived appendix samples fixed in formalin and embedded in p

In [ ]:
sentences1 = claims[1]['claim']
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
sentences2 = corpus[claims[1]['retrieved_doc_ids'][0]]['abstract']
embeddings2 = model.encode(sentences2, convert_to_tensor=True)
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
print(cosine_scores)

tensor([[0.2336, 0.2200, 0.3975, 0.4773, 0.5968, 0.2880, 0.4083, 0.4307, 0.1292]])


In [ ]:
print(retrieved_corpus)
print(claims)

{0: array(['10608397', '10607877'], dtype='<U9'), 2: array(['13734012', '17333231'], dtype='<U9'), 4: array(['7165938', '1387104'], dtype='<U9'), 6: array(['23117378', '2613775'], dtype='<U9'), 9: array(['44265107', '24700152'], dtype='<U9'), 10: array(['25510546', '13780287'], dtype='<U9'), 11: array(['25510546', '13780287'], dtype='<U9'), 12: array(['11705328', '43557480'], dtype='<U9'), 14: array(['1373287', '6421792'], dtype='<U9'), 15: array(['24766509', '8891333'], dtype='<U9'), 17: array(['7662395', '17450673'], dtype='<U9'), 18: array(['7165938', '1387104'], dtype='<U9'), 19: array(['36178047', '13072113'], dtype='<U9'), 20: array(['36178047', '13072113'], dtype='<U9'), 21: array(['41493639', '14021596'], dtype='<U9'), 22: array(['25451374', '35022568'], dtype='<U9'), 24: array(['1900152', '7975937'], dtype='<U9'), 25: array(['23117378', '2613775'], dtype='<U9'), 26: array(['4336849', '38410121'], dtype='<U9'), 27: array(['4336849', '38410121'], dtype='<U9'), 28: array(['280155

In [ ]:
number_of_datas =len(claims)
number_of_datas = 25

In [ ]:

cv = 0
claim_result_whole = []
for claim in claims:
  claims_result = {}
  sentences1 = claim['claim']
  embeddings1 = model.encode(sentences1, convert_to_tensor=True)

  #print(retrieved_corpus)
  claims_result['id'] = claim['id']
  claims_result['claim'] = claim['claim']
  claims_result['evidence'] = []

  relevant_ids = claim['id']

  for ids in retrieved_corpus[relevant_ids]:
    #print(ids)
    sentences2 = corpus[ids]['abstract']
    embeddings2 = model.encode(sentences2, convert_to_tensor=True)
    #Compute cosine-similarits
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    cos_sim = cosine_scores.numpy()[0]
    #cos_sim.append(cosine_scores.numpy())
    cos_sim[(cos_sim>=0.52) & (cos_sim<0.58)] = -1 # contradict
    #cos_sim[cos_sim<0.45] = 0 # neutral
    cos_sim[cos_sim>=0.58 ] = 1 # support
    cos_sim[(cos_sim !=1) & (cos_sim != -1)] = 0
    s_array = []
    c_array = []
    #print(cos_sim)
    for i in range(len(cos_sim)):
      if cos_sim[i] == 1:
        s_array.append(i)
      elif cos_sim[i] == -1:
        c_array.append(i)
    tmp = {}
    #print(corpus[ids]['doc_id'])
    
    
    #tmp[1]
    #tmp[corpus[ids]['doc_id']] = []
    if len(s_array) != 0:
      title = str(corpus[ids]['doc_id'])
      tmp[title] = []
      tmp[title].append({
        "sentences" : s_array,
        "label" : "SUPPORT"
      })
      #print(tmp)
      #claims_result['evidence'].append( tmp)
      print()
    if len(c_array) != 0:
      title = str(corpus[ids]['doc_id'])
      if not title in tmp:
        tmp[title] = []
      #print("tmp: ",tmp)
      #print("c_array: ",c_array)
      #print("claim_ r : ",claims_result) 
      tmp[title].append({
                "sentences" : c_array,
                "label" : "CONTRADICT"
            })
      #print("evidence elott tmp: ",tmp)
    if tmp :
      claims_result['evidence'].append(tmp)
      #print("claim_ r : ",claims_result) 
      #print()
    
      

  print(sentences1)
  #print(corpus[claims[0]['retrieved_doc_ids']['abstract'])
  print(claims_result)
  claim_result_whole.append(claims_result)
  print()
  cv +=1
  if cv == number_of_datas:
    break

0-dimensional biomaterials lack inductive properties.
{'id': 0, 'claim': '0-dimensional biomaterials lack inductive properties.', 'evidence': []}


1 in 5 million in UK have abnormal PrP positivity.
{'id': 2, 'claim': '1 in 5 million in UK have abnormal PrP positivity.', 'evidence': [{'13734012': [{'sentences': [4], 'label': 'SUPPORT'}]}]}



1-1% of colorectal cancer patients are diagnosed with regional or distant metastases.
{'id': 4, 'claim': '1-1% of colorectal cancer patients are diagnosed with regional or distant metastases.', 'evidence': [{'7165938': [{'sentences': [9], 'label': 'SUPPORT'}, {'sentences': [3, 4, 7, 8], 'label': 'CONTRADICT'}]}, {'1387104': [{'sentences': [1, 6, 8, 11], 'label': 'SUPPORT'}, {'sentences': [7], 'label': 'CONTRADICT'}]}]}



10% of sudden infant death syndrome (SIDS) deaths happen in newborns aged less than 6 months.
{'id': 6, 'claim': '10% of sudden infant death syndrome (SIDS) deaths happen in newborns aged less than 6 months.', 'evidence': [{'2311

In [ ]:
# print(claim_result_whole[1])
# print(claim_result_whole[1]["evidence"])
# for evidences_key in claim_result_whole[1]["evidence"][0]:
#   print(claim_result_whole[1]["evidence"][0])
#   print(claim_result_whole[1]["evidence"][0][evidences_key])
#   print(evidences_key)
#   for proof in claim_result_whole[1]["evidence"][0][evidences_key]:
#     print(proof['label'])

In [ ]:
# print(claims[1])
# print(len(claims[1]["evidence"]))
# for evidences_key in claims[1]["evidence"]:
#   print(claims[1]["evidence"][evidences_key])
#   print(evidences_key)
#   for proof in claims[1]["evidence"][evidences_key]:
#     print(proof['label'])

In [ ]:
len(claims)

809

In [ ]:
sum_egyezik = 0
sum_sum_egyezik=0
sum_support=0

con_con_egyezik=0
sum_contradic=0

false_pos = 0
false_neg =0

all_empty =0
empty_sum = 0

all_notempty = 0
# szam=0

#for i in range(len(claims)):
for i in range(number_of_datas):
  #print(i)
  #print(claims[i]["evidence"], "     ", claim_result_whole[i]["evidence"])
  if len(claims[i]["evidence"])==0:
    all_empty += 1
  else: 
    all_notempty +=1

  if len(claims[i]["evidence"])==0 and len(claim_result_whole[i]["evidence"])==0:
    #print("ures")
    sum_egyezik += 1
    empty_sum +=1
    continue 

  for evidences_key in claims[i]["evidence"]:
    for id_predected in claim_result_whole[i]["evidence"]:
      if evidences_key in id_predected:
         for proof in claims[i]["evidence"][evidences_key]:
           #print(claim_result_whole[i]["evidence"])
           for h in range(len(claim_result_whole[i]["evidence"])):
            #print(claim_result_whole[i]["evidence"][h], evidences_key)
            if evidences_key in claim_result_whole[i]["evidence"][h]:
              for proof_result in claim_result_whole[i]["evidence"][h][evidences_key]:
                if proof['label'] == proof_result["label"]:
                  sum_egyezik +=1
                if proof['label'] == 'SUPPORT':
                  sum_support+=1;
                  if proof_result['label'] == 'SUPPORT':
                    sum_sum_egyezik +=1
                  if proof_result['label'] == 'CONTRADICT':
                    false_pos +=1
                if proof['label'] == 'CONTRADICT':
                  sum_contradic+=1;
                  if proof_result['label'] == 'CONTRADICT':
                    con_con_egyezik +=1
                  if proof_result['label'] == 'SUPPORT':
                    false_neg +=1
      # else:
      #   szam +=1


print("uresek szama: ",empty_sum," / ", all_empty)
print("nem uresek szama:", all_notempty)
print(sum_egyezik, " / ", number_of_datas, " in percent: ", sum_egyezik/number_of_datas)
print("true support : ", sum_sum_egyezik, " / ", sum_support) # mennyiből mennyit találok el
print("true contradict : ",con_con_egyezik, " / ", sum_contradic) # mennyiből mennyit találok el
print("false pos: ", false_pos, " / ", sum_support) # mennyiből mennyit találok el
print("false neg: ", false_neg, " / ", sum_contradic) # mennyiből mennyit találok el
print("ami szerinte nem üres de szerintünk üres. ", all_notempty-(sum_contradic+sum_support)) # mennyiből mennyit találok el


uresek szama:  5  /  17
nem uresek szama: 8
7  /  25  in percent:  0.28
true support :  1  /  2
true contradict :  1  /  3
false pos:  1  /  2
false neg:  2  /  3
ami szerinte nem üres de szerintünk üres.  3


17 üres, 8 nem üres. \
17 üresból mi megtalálunk 4-et \
8- ból 5 les az amit classifikálva van és mi is mondunk rá valamit \
8-ból 3-nak ami szerinte nem üres de szerintünk üres. \
